## Index
- [Reading in data](#Reading-in-the-datasets)
- [Annotating table with relatedness information](#Annotating-table-with-relatedness-information)
- [Calculating statistics per population](#Calculating-statistics-per-population)
- [Formatting table for exporting](#Formatting-table-for-exporting)
- [Exporting final table](#Exporting-final-table)

### The purpose of this script is to format and write out a matrix table which will be used to create 'table 1' for our resource manuscript. 
#### This script contains information on how to:
- select specific columns from a matrix table
- annotate filter flags onto a matrix table
- join the columns of two matrix tables
- join two tables
- group a matrix table by region, population 
- use hl.agg.stats to calculate statics for a metric within a population
- count the number of samples where a filter flag equals True  

**Author: Zan Koenig**

In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 3.1.1
SparkUI available at http://znk-m.c.diverse-pop-seq-ref.internal:33345
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.77-684f32d73643
LOGGING: writing to /home/hail/hail-20211116-2136-0.2.77-684f32d73643.log


# Reading in data

In [2]:
# Reading in the post QC version of the merged dataset (with metadata)
### FIX: NEED TO UPDATE THIS PATH WITH THE NEWEST POST_QC MATRIX TABLE
mt = hl.read_matrix_table('gs://hgdp-1kg/hgdp_tgp/qc_and_figure_generation/new_hgdp_tgp_postQC.mt')

In [3]:
# Although sample_qc may already exist I am running it again to ensure it calculates metrics
# only for the samples currently in the postQC dataset
mt = hl.sample_qc(mt, name = "sample_qc")

In [4]:
# Grabbing only the columns from the matrix table (outputs table of just columns)
col_table = mt.cols()

2021-11-16 21:37:00 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


In [5]:
# Writing out a col table with only the columns needed for table 1
### FIX: IN THE PROCESS OF UPDATING FILE/FIELD PATHS WITH NEW GEOGRAPHIC REGION FIELD GENERATED
### IN THIS SECTION I NEED TO CHANGE COL_TABLE.HGDP_TGP_META.STUDY.REGION WITH COL_TABLE.STUDY_GEO_REGION
col_table = col_table.select(col_table.hgdp_tgp_meta.Study.region,
                             col_table.hgdp_tgp_meta.Population,
                             col_table.sample_qc.n_snp,
                             col_table.sample_qc.n_singleton,
                             col_table.bam_metrics.mean_coverage)

In [ ]:
print_count(col_table)

In [ ]:
# writing out col_table as a checkpoint to make the downstream steps run faster
# this is done because running sample_qc is computationally expensive
col_table.checkpoint('gs://hgdp-1kg/hgdp_tgp/qc_and_figure_generation/table_1_checkpoint.ht', overwrite=True)

In [8]:
# this is a table of only the columns with only postQC information
col_table = hl.read_table('gs://hgdp-1kg/hgdp_tgp/qc_and_figure_generation/table_1_checkpoint.ht')

In [9]:
# Since col_table is a table, count prints the number of rows which is equal to the number of samples
col_table.count()

4097

In [6]:
# hl.count() gives the number of rows in col_table which is equivalent to the number of samples
col_table.count()

4097

In [ ]:
print_count(col_table)

# Annotating table with relatedness info
This step is conducted so that [INSERT INFORMATION ON REASONING BEHIND RELATEDNESS FILTERING]

In [10]:
# Need to get number of unrelateds annotated to the table
# Reading in the unrelated and related matrix tables
unrelated = hl.read_matrix_table('gs://hgdp-1kg/hgdp_tgp/unrel_updated.mt')
related = hl.read_matrix_table('gs://hgdp-1kg/hgdp_tgp/rel_updated.mt')

# Annotating both the unrelated and the related tables with a flag named unrelated 
# set unrelated flag to True for those in the unrelated dataset, and False for those in the related dataset
unrelated = unrelated.annotate_cols(unrelated = True)
related = related.annotate_cols(unrelated = False)

# using hl.cols() to obtain two tables with only the columns from the original matrix tables
unrelated_cols = unrelated.cols()
related_cols = related.cols()

In [11]:
# Annotating the related/unrelated mts with counts per population
related_count = related_cols.aggregate(hl.agg.counter(related_cols.hgdp_tgp_meta.Population))
unrelated_count = unrelated_cols.aggregate(hl.agg.counter(unrelated_cols.hgdp_tgp_meta.Population))

print(f"Number of related individuals per population: {related_count}\n\nNumber of unrelated individuals per population: {unrelated_count}")

Number of related individuals per population: frozendict({'Naxi': 1, 'PJL': 48, 'Bedouin': 3, 'Pima': 3, 'IBS': 49, 'Mozabite': 1, 'Surui': 1, 'GWD': 60, 'PUR': 34, 'MbutiPygmy': 2, 'Hezhen': 1, 'She': 1, 'ITU': 4, 'BantuKenya': 2, 'Lahu': 3, 'Palestinian': 7, 'Maya': 3, 'STU': 16, 'ACB': 21, 'MXL': 34, 'GIH': 3, 'ASW': 19, 'Orcadian': 2, 'CEU': 55, 'Colombian': 4, 'Karitiana': 2, 'PEL': 37, 'Kalash': 2, 'CDX': 5, 'Makrani': 1, 'Druze': 7, 'CLM': 36, 'CHS': 60, 'BEB': 32, 'GBR': 2, 'Hazara': 4, 'BiakaPygmy': 4, 'YRI': 58, 'Melanesian': 2, 'KHV': 21, 'Japanese': 1, 'Mandenka': 3, 'MSL': 15, 'Sindhi': 1, 'LWK': 5, 'ESN': 45})

Number of unrelated individuals per population: frozendict({'Adygei': 17, 'Naxi': 8, 'PJL': 97, 'Tuscan': 8, 'Bedouin': 43, 'Pima': 11, 'BantuSouthAfrica': 8, 'IBS': 107, 'Italian': 11, 'Papuan': 17, 'Mozabite': 27, 'Surui': 7, 'Mongola': 10, 'Russian': 25, 'Basque': 23, 'FIN': 98, 'Sardinian': 27, 'GWD': 116, 'PUR': 104, 'MbutiPygmy': 12, 'Hezhen': 8, 'She': 9, 'I

In [12]:
# Joining the columns of the unrelated and related datasets
mt_rel = unrelated.union_cols(related)

# counting the number of unrelated in the matrix table to make sure it is as expected
mt_rel.aggregate_cols(hl.agg.counter(mt_rel.unrelated))

# creating a table with only the columns from the matrix table containing related information
# this is done since the final output will be a tsv and thus must be in table format
# Being a table of columns allows it to be annotated onto the existing col_table as shown below
rel_table = mt_rel.cols()

# annotating the relatedness information onto the column table
col_table = col_table.annotate(unrel = rel_table[col_table.s].unrelated)

# Calculating statistics per population
In this section, we will be using hl.agg.stats() which calculates the following metrics for a given expression:
   - min
   - max
   - mean
   - standard deviation
   - number of non-missing records
   - sum

Using hl.group_by() we calculate these statistics for each of the 78 populations in this dataset.
We also use hl.agg.count_where() to count where the field denoting if samples are related or not is True which will give us the number of related samples within each population.

In [13]:
# Calculating stats per population for each metric grouped by geographic region and population
table = n_snp = col_table.group_by(
    col_table.region, col_table.Population).aggregate(
    n_snp_stats = hl.agg.stats(col_table.n_snp),
    n_singleton_stats = hl.agg.stats(col_table.n_singleton),
    cov_stats = hl.agg.stats(col_table.mean_coverage),
    n_unrelated = hl.agg.count_where(col_table.unrel == True))

In [14]:
# hl.show() prints 10 lines or however many lines specified of a matrix table or table
# running it here to check that each of the table field contain what we'd expect
table.show()

2021-11-16 22:11:09 Hail: INFO: Coerced sorted dataset
2021-11-16 22:11:11 Hail: INFO: Ordering unsorted dataset with network shuffle


+--------+------------+------------------+-------------------+-----------------+
| region | Population | n_snp_stats.mean | n_snp_stats.stdev | n_snp_stats.min |
+--------+------------+------------------+-------------------+-----------------+
| str    | str        |          float64 |           float64 |         float64 |
+--------+------------+------------------+-------------------+-----------------+
| "AFR"  | "ACB"      |         6.04e+06 |          6.81e+04 |        5.73e+06 |
| "AFR"  | "ASW"      |         5.94e+06 |          9.38e+04 |        5.67e+06 |
| "AFR"  | "ESN"      |         6.13e+06 |          1.46e+04 |        6.09e+06 |
| "AFR"  | "GWD"      |         6.12e+06 |          1.95e+04 |        6.06e+06 |
| "AFR"  | "LWK"      |         6.11e+06 |          1.72e+04 |        6.06e+06 |
| "AFR"  | "MSL"      |         6.18e+06 |          1.51e+04 |        6.13e+06 |
| "AFR"  | "YRI"      |         6.13e+06 |          1.50e+04 |        6.08e+06 |
| "AMR"  | "CLM"      |         5.29e+06 |          7.87e+04 |        5.17e+06 |
| "AMR"  | "MXL"      |         5.27e+06 |          3.14e+04 |        5.18e+06 |
| "AMR"  | "PEL"      |         5.29e+06 |          5.00e+04 |        5.23e+06 |
+--------+------------+------------------+-------------------+-----------------+

+-----------------+---------------+-----------------+------------------------+
| n_snp_stats.max | n_snp_stats.n | n_snp_stats.sum | n_singleton_stats.mean |
+-----------------+---------------+-----------------+------------------------+
|         float64 |         int64 |         float64 |                float64 |
+-----------------+---------------+-----------------+------------------------+
|        6.14e+06 |           114 |        6.88e+08 |               1.57e+04 |
|        6.10e+06 |            71 |        4.22e+08 |               1.40e+04 |
|        6.18e+06 |           148 |        9.07e+08 |               8.56e+03 |
|        6.17e+06 |           176 |        1.08e+09 |               1.04e+04 |
|        6.16e+06 |            97 |        5.93e+08 |               2.58e+04 |
|        6.22e+06 |            98 |        6.06e+08 |               2.13e+04 |
|        6.16e+06 |           175 |        1.07e+09 |               8.38e+03 |
|        5.58e+06 |           130 |        6.87e+08 |               8.88e+03 |
|        5.34e+06 |            97 |        5.11e+08 |               9.27e+03 |
|        5.60e+06 |           122 |        6.45e+08 |               1.17e+04 |
+-----------------+---------------+-----------------+------------------------+

+-------------------------+-----------------------+-----------------------+
| n_singleton_stats.stdev | n_singleton_stats.min | n_singleton_stats.max |
+-------------------------+-----------------------+-----------------------+
|                 float64 |               float64 |               float64 |
+-------------------------+-----------------------+-----------------------+
|                8.38e+03 |              5.28e+02 |              2.75e+04 |
|                8.23e+03 |              5.32e+02 |              2.76e+04 |
|                5.68e+03 |              5.49e+02 |              2.00e+04 |
|                6.80e+03 |              5.73e+02 |              2.60e+04 |
|                7.78e+03 |              9.39e+03 |              3.98e+04 |
|                8.86e+03 |              7.02e+02 |              3.11e+04 |
|                5.64e+03 |              5.18e+02 |              2.30e+04 |
|                6.56e+03 |              2.84e+02 |              2.62e+04 |
|                7.25e+03 |              2.95e+02 |              2.83e+04 |
|                8.64e+03 |              2.34e+02 |              3.15e+04 |
+-------------------------+-----------------------+-----------------------+

+---------------------+-----------------------+----------------+
| n_singleton_stats.n | n_singleton_stats.sum | cov_stats.mean |
+---------------------+-----------------------+----------------+
|               int64 |   

# Formatting table for exporting
In this section

In [21]:
# Flattening out the structs created from annotating the tables
# this is done so that when the able is written out, the data is easier to work with
# if the tables were written out without flattening the new annotated information would be in a nested structure
table = table.flatten()

table = table.key_by(table.region, table.Population)

In [17]:
table.show()

2021-06-29 16:31:23 Hail: INFO: Coerced sorted dataset
2021-06-29 16:31:24 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-06-29 16:31:25 Hail: INFO: Coerced sorted dataset
2021-06-29 16:31:25 Hail: INFO: Coerced sorted dataset
2021-06-29 16:31:26 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-06-29 16:31:27 Hail: INFO: Coerced sorted dataset
2021-06-29 16:31:28 Hail: INFO: Coerced sorted dataset
2021-06-29 16:31:29 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-06-29 16:31:29 Hail: INFO: Coerced sorted dataset


,,,,,,,,,,,,,
region,Population,n_unrelated,cov_stats.n,cov_stats.mean,cov_stats.stdev,n_snp.n_unrelated,n_snp.n_snp_stats.n,n_snp.n_snp_stats.mean,n_snp.n_snp_stats.stdev,n_singleton.n_unrelated,n_singleton.n_singleton_stats.n,n_singleton.n_singleton_stats.mean,n_singleton.n_singleton_stats.stdev
str,str,int64,int64,float64,float64,int64,int64,float64,float64,int64,int64,float64,float64
"""AFR""","""ACB""",90,114,3.17e+01,2.48e+00,90,114,6.04e+06,6.81e+04,90,114,1.57e+04,8.38e+03
"""AFR""","""ASW""",49,72,3.23e+01,3.34e+00,49,72,5.94e+06,1.06e+05,49,72,1.39e+04,8.04e+03
"""AFR""","""ESN""",100,148,3.23e+01,3.23e+00,100,148,6.13e+06,1.46e+04,100,148,8.56e+03,5.68e+03
"""AFR""","""GWD""",112,176,3.25e+01,2.71e+00,112,176,6.12e+06,1.95e+04,112,176,1.04e+04,6.80e+03
"""AFR""","""LWK""",92,97,3.29e+01,4.32e+00,92,97,6.11e+06,1.72e+04,92,97,2.58e+04,7.77e+03
"""AFR""","""MSL""",80,98,3.18e+01,2.69e+00,80,98,6.18e+06,1.51e+04,80,98,2.13e+04,8.86e+03
"""AFR""","""YRI""",114,175,3.23e+01,3.59e+00,114,175,6.13e+06,1.50e+04,114,175,8.38e+03,5.64e+03
"""AMR""","""CLM""",91,130,3.25e+01,3.14e+00,91,130,5.29e+06,7.87e+04,91,130,8.88e+03,6.56e+03


In [24]:
# Checking on the format of the table after flattening to make sure it is what we expect
table.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'region': str 
    'Population': str 
    'n_snp_stats.mean': float64 
    'n_snp_stats.stdev': float64 
    'n_snp_stats.min': float64 
    'n_snp_stats.max': float64 
    'n_snp_stats.n': int64 
    'n_snp_stats.sum': float64 
    'n_singleton_stats.mean': float64 
    'n_singleton_stats.stdev': float64 
    'n_singleton_stats.min': float64 
    'n_singleton_stats.max': float64 
    'n_singleton_stats.n': int64 
    'n_singleton_stats.sum': float64 
    'cov_stats.mean': float64 
    'cov_stats.stdev': float64 
    'cov_stats.min': float64 
    'cov_stats.max': float64 
    'cov_stats.n': int64 
    'cov_stats.sum': float64 
    'n_unrelated': int64 
----------------------------------------
Key: []
----------------------------------------


In [25]:
# one last sanity check before writing out the dataset to make sure you still have the number of rows you expect
# in this case, since the data is grouped by global region, population
# the number of rows should be equal to the number of populations
table.count()

2021-11-16 22:20:22 Hail: INFO: Ordering unsorted dataset with network shuffle


78

# Exporting final table

In [26]:
# writing out the final table x tsv
table.export('gs://hgdp-1kg/hgdp_tgp/qc_and_figure_generation/table_1.tsv', header=True)

2021-11-16 22:20:50 Hail: INFO: Coerced sorted dataset
2021-11-16 22:20:51 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-11-16 22:20:53 Hail: INFO: merging 16 files totalling 14.9K...
2021-11-16 22:20:53 Hail: INFO: while writing:
    gs://hgdp-1kg/hgdp_tgp/qc_and_figure_generation/table_x.tsv
  merge time: 413.016ms
